In [2]:
# to handle dataset
import pandas as pd
import datetime as dt

#to visualize
import matplotlib.pyplot as plt
%matplotlib inline

#to crawl stock data
import pandas_datareader.data as web

In [ ]:
start = dt.datetime(2019,1,1)
end = dt.datetime.now() 

bbca = web.DataReader("BBCA.JK", 'yahoo', start,end)
dji = web.DataReader("^DJI",'yahoo',start,end)
nikkei = web.DataReader("^N225", start, end)
sp500 = web.DataReader("^GSPC", start, end)
hsi = web.DataReader("^HSI", start, end)